# Transfer Learning with Tensorflow Part 1: Feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problems.
There are two main benefits:
1. Can leverage an existing neural network architecture proven to work on problems similat to our own.
2. Can leverage a working neural network achitecture which has already learned patterns to similar data to our own, then we can adapt those patterns to our own data

### Importing the data

In [1]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("../data/10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in : {dirpath}")

There are 2 directories and 0 images in : ../data/10_food_classes_10_percent
There are 10 directories and 0 images in : ../data/10_food_classes_10_percent\test
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\test\chicken_curry
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\test\chicken_wings
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\test\fried_rice
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\test\grilled_salmon
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\test\hamburger
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\test\ice_cream
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\test\pizza
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\test\ramen
There are 0 directories and 250 images in : ../data/10_food_classes_10_percent\tes

### Creating the data loaders (preparing the data)
We'll use the ImageDataGenerator class to load in our images in batches

In [3]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "../data/10_food_classes_10_percent/train/"
test_dir = "../data/10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training Images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")
print("Testing images:")
test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")


Training Images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


### Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality we can add to our models to be performed during or after training.
Some of the most popular callbacks are:
* Tracking experiments with the TensorBoard callback.
* Model checkpoint with the modelCheckpoint callback.
* Stopping a model from training (before it trains too long and overfits) with the Early Stopping callback.

In [4]:
# Create TensorBoard callback (functionized because we need to create  a new one for each model)
import datetime
import tensorflow as tf
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving Tensorboard log files to: {log_dir}")
    return tensorboard_callback

### Creating models using TensorFlow Hub

In the pas we've used TensorFlow to create our own models layers by layer from scratch.

Now we're going to do a similar process, except majority of our model's layers are going to come from Tensorflow hub.

we can access pretrained models on tfhub.dev

In [5]:
# Let's compare the following two models:
resnet_url = "https://www.kaggle.com/models/google/resnet-v2/TensorFlow2/101-feature-vector/2"

efficientnet_url = "https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-feature-vector/1"

In [6]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from keras import layers

In [7]:
# Let's make a create_model() function to create a model from a URL
def create_model(model_url, num_classes=10):
    """
    Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

    Args:
        model_url (str): A Tensorflow hub/Kaggle feature extraction URL
        num_classes (int): Number of output neurons in the output layer. Default 10.

    Returns:
        An uncompiled Keras Sequential model with model_url as a feature extractor layer and
        Dense output layer with num_classes output neurons
    """

    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url, trainable=False, name="Feature_Extraction_Layer",
                                             input_shape=IMAGE_SHAPE+(3,)) # freeze the already learned patterns with trainable false

    # Create the model.
    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation="softmax", name="Output_Layer")
    ])

    return model

### Creating ResNet TensorFlow hub feature extraction model

In [10]:
# Create ResNet Model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

In [11]:
# Compile the model
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [13]:
# Let's fit our resnet model to the data.
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data_10_percent,
                                  validation_steps=len(test_data_10_percent),
                                  callbacks=[create_tensorboard_callback(dir_name="../tensorflow_hub",
                                                                         experiment_name="resnetv2")])

Saving Tensorboard log files to: ../tensorflow_hub/resnetv2/20251225-151741
Epoch 1/5
24/24 [==============================] - 70s 3s/step - loss: 0.2594 - accuracy: 0.9547 - val_loss: 0.6096 - val_accuracy: 0.7996
Epoch 2/5
24/24 [==============================] - 69s 3s/step - loss: 0.2124 - accuracy: 0.9680 - val_loss: 0.6049 - val_accuracy: 0.8004
Epoch 3/5
24/24 [==============================] - 98s 4s/step - loss: 0.1768 - accuracy: 0.9800 - val_loss: 0.5889 - val_accuracy: 0.8012
Epoch 4/5
24/24 [==============================] - 73s 3s/step - loss: 0.1492 - accuracy: 0.9880 - val_loss: 0.5843 - val_accuracy: 0.8064
Epoch 5/5
24/24 [==============================] - 72s 3s/step - loss: 0.1269 - accuracy: 0.9920 - val_loss: 0.5824 - val_accuracy: 0.8056


### Creating and Testing EfficientNetB0 TensorFlow Hub Feature  Extraction Model

In [14]:
# Create EfficientNetB0 feature extractor model
efficientnet_model = create_model(model_url=efficientnet_url,
                                  num_classes=train_data_10_percent.num_classes)

# Compile the model
efficientnet_model.compile(loss="categorical_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

# Fit the model to our 10% of training data.
efficientnet_history = efficientnet_model.fit(train_data_10_percent,
                                              epochs=5,
                                              steps_per_epoch=len(train_data_10_percent),
                                              validation_data=test_data_10_percent,
                                              validation_steps=len(test_data_10_percent),
                                              callbacks=[create_tensorboard_callback(dir_name="../tensorflow_hub",
                                                                         experiment_name="efficientnetb0")])

Saving Tensorboard log files to: ../tensorflow_hub/efficientnetb0/20251225-153358
Epoch 1/5
24/24 [==============================] - 21s 731ms/step - loss: 1.8393 - accuracy: 0.4093 - val_loss: 1.2859 - val_accuracy: 0.7256
Epoch 2/5
24/24 [==============================] - 16s 699ms/step - loss: 1.0622 - accuracy: 0.7547 - val_loss: 0.8594 - val_accuracy: 0.8144
Epoch 3/5
24/24 [==============================] - 16s 699ms/step - loss: 0.7606 - accuracy: 0.8227 - val_loss: 0.6909 - val_accuracy: 0.8440
Epoch 4/5
24/24 [==============================] - 17s 711ms/step - loss: 0.6133 - accuracy: 0.8587 - val_loss: 0.6061 - val_accuracy: 0.8540
Epoch 5/5
24/24 [==============================] - 16s 698ms/step - loss: 0.5153 - accuracy: 0.8827 - val_loss: 0.5516 - val_accuracy: 0.8640
